In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.patches import Ellipse, Arc, Circle, PathPatch
from matplotlib.path import Path


# ----------------------------
# Color helpers (ALL COLORS: 0..255 RGB ONLY)
# ----------------------------
def _rgb_to_mpl(rgb):
    """
    Accept RGB as either:
      - 0..255 integers (preferred)
      - 0..1 floats (allowed ONLY if at least one channel is a non-integer float)

    This avoids ambiguity for (0,0,0) and (1,1,1).
    """
    if rgb is None:
        return None
    if len(rgb) != 3:
        raise ValueError(f"RGB must be a 3-tuple, got: {rgb}")

    r, g, b = rgb

    # Decide scale:
    # If any channel is a non-integer float in [0,1], treat as 0..1
    def is_nonint_float(x):
        return isinstance(x, float) and not float(x).is_integer()

    looks_like_unit_float = (
        all(0 <= v <= 1 for v in (r, g, b)) and any(is_nonint_float(v) for v in (r, g, b))
    )

    if looks_like_unit_float:
        return (float(r), float(g), float(b))

    # Otherwise treat as 0..255
    for v in (r, g, b):
        if v < 0 or v > 255:
            raise ValueError(f"RGB values must be in [0,255] (or 0..1 floats), got: {rgb}")

    return (float(r) / 255.0, float(g) / 255.0, float(b) / 255.0)


# ----------------------------
# Geometry sampling helpers
# ----------------------------
def _sample_ellipse(cx, cy, w, h, n=2000):
    rx, ry = w / 2.0, h / 2.0
    t = np.linspace(0, 2 * np.pi, n, endpoint=False)
    return np.c_[cx + rx * np.cos(t), cy + ry * np.sin(t)]


def _sample_arc(cx, cy, w, h, t1, t2, n=800):
    rx, ry = w / 2.0, h / 2.0
    th = np.deg2rad(np.linspace(t1, t2, n))
    return np.c_[cx + rx * np.cos(th), cy + ry * np.sin(th)]


def _path_between_indices(pts, i, j):
    """Return both possible paths between i and j along a closed point ring."""
    def forward(a, b):
        return pts[a:b + 1] if a <= b else np.vstack([pts[a:], pts[:b + 1]])
    p1 = forward(i, j)
    p2 = forward(j, i)[::-1]
    return p1, p2


def _side_path_between_indices(pts, i, j, side="left"):
    """
    Choose the path between indices that lies more on the left (smaller mean x)
    or right (larger mean x) side of the shape.
    """
    p1, p2 = _path_between_indices(pts, i, j)
    if side == "left":
        return p1 if p1[:, 0].mean() <= p2[:, 0].mean() else p2
    if side == "right":
        return p1 if p1[:, 0].mean() >= p2[:, 0].mean() else p2
    raise ValueError("side must be 'left' or 'right'")


def _top_path_between_indices(pts, i, j):
    """Choose the path that is 'top' (higher mean y)."""
    def forward(a, b):
        return pts[a:b + 1] if a <= b else np.vstack([pts[a:], pts[:b + 1]])
    p1 = forward(i, j)
    p2 = forward(j, i)[::-1]
    return p1 if p1[:, 1].mean() >= p2[:, 1].mean() else p2


# ----------------------------
# Main drawing function
# ----------------------------
def draw_face_outline(
    out_path="face_only.png",
    size_px=700,
    lw=20,

    # OUTLINE COLORS
    face_color=(0, 0, 0),
    brain_color=(0, 0, 0),

    # Ear OUTLINE (stroke) colors (optional). If None, defaults to face_color.
    ear_left_color=None,
    ear_right_color=None,

    # Ear INSIDE fill colors (region between ear arc and face outline). If None, no fill.
    ear_left_fill_color=None,
    ear_right_fill_color=None,

    # FILL COLORS
    face_fill_color=(255, 255, 255),
    helmet_fill_color=None,

    # WINKS
    wink_left=False,
    wink_right=False,

    # EYE COLORS
    # (eye = eyeball fill, pupil = iris fill; pupil center is controlled separately)
    color_left_eye=(255, 255, 255),
    color_left_pupil=(0, 200, 255),
    color_right_eye=(255, 255, 255),
    color_right_pupil=(0, 200, 255),

    # NEW: pupil center color per eye (black by default)
    color_left_pupil_center=(0, 0, 0),
    color_right_pupil_center=(0, 0, 0),

    # Eye highlight dot color (white by default)
    eye_highlight_color=(255, 255, 255),

    pupil_border=True,
    pupil_border_lw=2.0,

    open_eye_drop=0.018,

    # Brain nodes
    node_rgbs=None,          # list of RGB tuples (0..255)
    node_positions=None,     # list of (x,y)
    node_radius=0.028,

    show=True,
):
    dpi = 100
    fig, ax = plt.subplots(figsize=(size_px / dpi, size_px / dpi), dpi=dpi)

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect("equal")
    ax.axis("off")

    # Convert colors once
    face_col = _rgb_to_mpl(face_color)
    brain_col = _rgb_to_mpl(brain_color)
    face_fill = _rgb_to_mpl(face_fill_color)
    helmet_fill = _rgb_to_mpl(helmet_fill_color)

    ear_left_col = _rgb_to_mpl(ear_left_color) if ear_left_color is not None else face_col
    ear_right_col = _rgb_to_mpl(ear_right_color) if ear_right_color is not None else face_col
    ear_left_fill = _rgb_to_mpl(ear_left_fill_color)
    ear_right_fill = _rgb_to_mpl(ear_right_fill_color)

    left_eye_fill = _rgb_to_mpl(color_left_eye)
    left_iris_fill = _rgb_to_mpl(color_left_pupil)
    right_eye_fill = _rgb_to_mpl(color_right_eye)
    right_iris_fill = _rgb_to_mpl(color_right_pupil)

    left_pupil_center_fill = _rgb_to_mpl(color_left_pupil_center)
    right_pupil_center_fill = _rgb_to_mpl(color_right_pupil_center)
    highlight_fill = _rgb_to_mpl(eye_highlight_color)

    # Geometry
    face_cx, face_cy = 0.5, 0.5
    face_w, face_h = 0.68, 0.78

    sep_cx, sep_cy = 0.5, 0.70
    sep_w, sep_h = 0.80, 0.40
    sep_t1, sep_t2 = 199, 341

    # Pre-sample face outline points (used for helmet fill and ear fills)
    face_pts = _sample_ellipse(face_cx, face_cy, face_w, face_h, 3000)

    # -------------------
    # FACE FILL
    # -------------------
    ax.add_patch(Ellipse(
        (face_cx, face_cy), face_w, face_h,
        facecolor=face_fill, edgecolor="none", zorder=0
    ))

    # Clip path to guarantee no helmet spill
    face_clip = Ellipse((face_cx, face_cy), face_w, face_h, transform=ax.transData)

    # -------------------
    # HELMET FILL (clipped)
    # -------------------
    if helmet_fill is not None:
        arc_pts = _sample_arc(sep_cx, sep_cy, sep_w, sep_h, sep_t1, sep_t2, 1200)

        i_l = int(np.argmin(((face_pts - arc_pts[0]) ** 2).sum(1)))
        i_r = int(np.argmin(((face_pts - arc_pts[-1]) ** 2).sum(1)))

        top_face = _top_path_between_indices(face_pts, i_l, i_r)
        helmet_poly = np.vstack([top_face, arc_pts[::-1]])

        helmet_patch = PathPatch(
            Path(helmet_poly, closed=True),
            facecolor=helmet_fill,
            edgecolor="none",
            zorder=1
        )
        helmet_patch.set_clip_path(face_clip)
        ax.add_patch(helmet_patch)

    # -------------------
    # EAR INSIDE FILLS
    # -------------------
    def add_ear_fill(side):
        if side == "left":
            fill_col = ear_left_fill
            if fill_col is None:
                return
            ear_arc_pts = _sample_arc(0.18, 0.47, 0.22, 0.28, 90, 270, 900)
            i_top = int(np.argmin(((face_pts - ear_arc_pts[0]) ** 2).sum(1)))
            i_bot = int(np.argmin(((face_pts - ear_arc_pts[-1]) ** 2).sum(1)))
            face_side = _side_path_between_indices(face_pts, i_top, i_bot, side="left")
        else:
            fill_col = ear_right_fill
            if fill_col is None:
                return
            ear_arc_pts = _sample_arc(0.82, 0.47, 0.22, 0.28, -90, 90, 900)
            i_top = int(np.argmin(((face_pts - ear_arc_pts[0]) ** 2).sum(1)))
            i_bot = int(np.argmin(((face_pts - ear_arc_pts[-1]) ** 2).sum(1)))
            face_side = _side_path_between_indices(face_pts, i_top, i_bot, side="right")

        ear_poly = np.vstack([ear_arc_pts, face_side[::-1]])
        ax.add_patch(PathPatch(
            Path(ear_poly, closed=True),
            facecolor=fill_col,
            edgecolor="none",
            zorder=2  # under outlines, above base face fill
        ))

    add_ear_fill("left")
    add_ear_fill("right")

    # -------------------
    # OUTLINES
    # -------------------
    ax.add_patch(Ellipse(
        (face_cx, face_cy), face_w, face_h,
        fill=False, edgecolor=face_col, linewidth=lw, zorder=10
    ))
    ax.add_patch(Arc(
        (sep_cx, sep_cy), sep_w, sep_h,
        theta1=sep_t1, theta2=sep_t2,
        linewidth=lw, color=face_col, zorder=10
    ))

    # EARS (outline strokes) — extend angles a bit and draw under the face outline
    ear_overlap_deg = 5  # try 4–10
    
    left_ear = Arc(
        (0.18, 0.47), 0.22, 0.28,
        theta1=99 - ear_overlap_deg, theta2=270 + ear_overlap_deg,
        linewidth=lw, color=ear_left_col, zorder=9  # <-- below face outline (which is zorder=10)
    )
    right_ear = Arc(
        (0.82, 0.47), 0.22, 0.28,
        theta1=-90 - ear_overlap_deg, theta2=81 + ear_overlap_deg,
        linewidth=lw, color=ear_right_col, zorder=9
    )
    
    # optional: nicer stroke ends at thin linewidths
    left_ear.set_capstyle("round")
    right_ear.set_capstyle("round")
    
    ax.add_patch(left_ear)
    ax.add_patch(right_ear)

    lw_face = lw * 0.55

    # -------------------
    # EYES
    # -------------------
    closed_eye_w, closed_eye_h = 0.16, 0.10
    closed_eye_y = 0.43
    eye_dx = 0.12

    open_eye_y = closed_eye_y - open_eye_drop
    open_eye_r = 0.045
    pupil_r = 0.018

    def draw_open_eye(center, eye_fill, iris_fill, pupil_center_fill):
        # outer eye (eyeball)
        ax.add_patch(Circle(
            center, open_eye_r,
            facecolor=eye_fill,
            edgecolor=face_col,
            linewidth=lw_face,
            zorder=11
        ))

        # iris
        iris_r = pupil_r * 1.9
        ax.add_patch(Circle(
            center, iris_r,
            facecolor=iris_fill,
            edgecolor="none",
            zorder=12
        ))

        # pupil center
        pupil_center_r = pupil_r * 0.9
        ax.add_patch(Circle(
            center, pupil_center_r,
            facecolor=pupil_center_fill,
            edgecolor=("black" if pupil_border else "none"),
            linewidth=pupil_border_lw if pupil_border else 0.0,
            zorder=13
        ))

        # highlight (tiny dot, top-right)
        highlight_offset = pupil_r * 0.6
        highlight_r = pupil_r * 0.25
        ax.add_patch(Circle(
            (center[0] + highlight_offset, center[1] + highlight_offset),
            highlight_r,
            facecolor=highlight_fill,
            edgecolor="none",
            zorder=14
        ))

    # Left eye
    left_center_closed = (0.5 - eye_dx, closed_eye_y)
    left_center_open = (0.5 - eye_dx, open_eye_y)
    if wink_left:
        ax.add_patch(Arc(
            left_center_closed, closed_eye_w, closed_eye_h,
            theta1=205, theta2=335,
            linewidth=lw_face, color=face_col, zorder=10
        ))
    else:
        draw_open_eye(left_center_open, left_eye_fill, left_iris_fill, left_pupil_center_fill)

    # Right eye
    right_center_closed = (0.5 + eye_dx, closed_eye_y)
    right_center_open = (0.5 + eye_dx, open_eye_y)
    if wink_right:
        ax.add_patch(Arc(
            right_center_closed, closed_eye_w, closed_eye_h,
            theta1=205, theta2=335,
            linewidth=lw_face, color=face_col, zorder=10
        ))
    else:
        draw_open_eye(right_center_open, right_eye_fill, right_iris_fill, right_pupil_center_fill)

    # -------------------
    # MOUTH
    # -------------------
    ax.add_patch(Arc(
        (0.5, 0.33), 0.26, 0.18,
        theta1=215, theta2=325,
        linewidth=lw_face, color=face_col, zorder=10
    ))

    # -------------------
    # BRAIN WIRES
    # -------------------
    lw_brain = lw * 0.55
    wires = [
        ([0.46, 0.46], [0.89, 0.72]),
        ([0.54, 0.54], [0.89, 0.72]),
        ([0.60, 0.60], [0.87, 0.75]),
        ([0.60, 0.64], [0.75, 0.75]),
        ([0.64, 0.64], [0.75, 0.68]),
    ]
    for x, y in wires:
        ax.add_line(mlines.Line2D(
            x, y,
            lw=lw_brain,
            color=brain_col,
            solid_capstyle="round",
            zorder=11
        ))

    # -------------------
    # NODES
    # -------------------
    if node_rgbs is None:
        node_rgbs = [(255, 0, 0), (0, 255, 0), (255, 200, 0)]
    positions = node_positions or [(0.46, 0.72), (0.54, 0.72), (0.64, 0.68)]

    if len(node_rgbs) != len(positions):
        raise ValueError(
            f"node_rgbs length ({len(node_rgbs)}) must match number of nodes ({len(positions)})."
        )

    for (x, y), rgb in zip(positions, node_rgbs):
        ax.add_patch(Circle(
            (x, y), node_radius,
            facecolor=_rgb_to_mpl(rgb),
            edgecolor=face_col,
            linewidth=lw * 0.45,
            zorder=12
        ))

    if show:
        plt.show()
    else:
        plt.savefig(out_path, bbox_inches="tight", pad_inches=0.02)
        plt.close(fig)
        print(f"Saved: {out_path}")



In [2]:
# default pallette
OUTLINE_BLACK = (25, 25, 25)        # softer than pure black
FACE_WHITE    = (250, 250, 248)     # warm off-white
HELMET_GRAY   = (235, 236, 238)     # subtle neutral helmet
EYE_WHITE     = (248, 248, 248)

IRIS_BLUE     = (72, 155, 186)      # muted cyan-blue
IRIS_GREEN    = (86, 155, 120)      # desaturated green

PUPIL_BLACK  = (30, 30, 30)
HIGHLIGHT    = (255, 255, 255)
EAR_PINK      = (255, 255, 255)
EAR_BLUE     = (255, 255, 255)
NODE_RED     = (198, 82, 82)
NODE_GREEN   = (88, 160, 112)
NODE_YELLOW  = (214, 178, 92)
NODE_PURPLE  = (148, 104, 186)
NODE_BASE     = (120, 148, 165)  # muted
NODE_HIGHLIGHT= (72, 155, 186)   # your iris blue, ties the system together

node_rgbs=[NODE_BASE, NODE_BASE, NODE_BASE, NODE_HIGHLIGHT]

draw_face_outline(
    out_path="face_only.png",
    size_px=700,
    lw=15,

    # outlines
    face_color=(25, 25, 25),
    brain_color=(25, 25, 25),

    # ears (outline)
    ear_left_color=None,
    ear_right_color=None,

    # ears (inside)
    ear_left_fill_color=EAR_BLUE,
    ear_right_fill_color=EAR_BLUE,

    # fills
    face_fill_color=(250, 250, 248),
    helmet_fill_color=(235, 236, 238),

    # expression
    wink_left=False,
    wink_right=True,

    # eyes
    color_left_eye=(248, 248, 248),
    color_right_eye=(248, 248, 248),

    color_left_pupil=(86, 155, 120),     # muted green
    color_right_pupil=(72, 155, 186),    # muted blue

    color_left_pupil_center=(30, 30, 30),
    color_right_pupil_center=(30, 30, 30),

    eye_highlight_color=(255, 255, 255),

    pupil_border=True,
    pupil_border_lw=1.8,

    open_eye_drop=0.018,

    node_rgbs=node_rgbs,
    
    node_positions=[(0.46, 0.72), (0.54, 0.72), (0.64, 0.68), (0.60, 0.80)],
    node_radius=0.026,

    show=False,
)


Saved: face_only.png
